# 📁 New Repository Structure

**Configuration files have been moved to be more accessible:**

- **Tool Config**: `config/tools.yaml` (was `src/config/tools/config.yaml`)
- **MCP Servers**: `config/mcp_servers/*.json` (was `src/mcp_integration/servers/*.json`)

**Benefits:**
- ✅ Easy to find at project root level
- ✅ Clear separation of config vs code
- ✅ Auto-discovery still works

**To view available tools:**
```bash
python -m src.config.tools.registry
```

---


# Custom Agent Creation Tutorial

This notebook teaches you how to build a custom LangGraph agent from scratch.

## 📚 What You'll Learn
1. Load and configure tools
2. Create and bind LLM with tools
3. Define agent state and logic
4. Build LangGraph workflow
5. Add memory with PostgreSQL checkpointer
6. Test your custom agent
7. Customize agent behavior

**Last Updated**: January 2025

## Setup

In [3]:
import sys
import os

# Add project root to path
project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
os.chdir(project_root)

print(f"✅ Project root: {project_root}")
print(f"✅ Current directory: {os.getcwd()}")

✅ Project root: /home/shamaseen/Desktop/Projects/personal/Langchain
✅ Current directory: /home/shamaseen/Desktop/Projects/personal/Langchain


In [4]:
# VERIFY_TOOLLOADER_AND_IMPORTS
from src.tools import ToolLoader
from src.mcp_integration.protocol import MCPTool

loader = ToolLoader()
tools = loader.get_tools()
print(f'Loaded {len(tools)} tools: {[t.name for t in tools]}')


Loaded 8 tools: ['gmail', 'calendar', 'cv_sheet_manager', 'process_cvs', 'search_candidates', 'search_create_sheet', 'datetime', 'webex']


## Step 1: Load Tools

First, let's load the tools our agent will use

In [5]:
from src.agents.tool_factory import get_tools

print("Step 1: Loading Tools\n" + "="*60)

# Get all available tools
tools = get_tools()

print(f"✅ Loaded {len(tools)} tool(s)")
print("\nTool details:")
for i, tool in enumerate(tools, 1):
    if hasattr(tool, 'name'):
        print(f"  {i}. {tool.name}")
        if hasattr(tool, 'description'):
            print(f"     {tool.description[:80]}...")
    else:
        print(f"  {i}. {type(tool).__name__}")

print("\n" + "="*60)
print("✅ Tools loaded successfully!")

Step 1: Loading Tools
🔧 Loading tools from dynamic configuration
   Config: src/config/tools.yaml

✅ Loaded 8 tools
   Active MCP clients: None
   Tool names: gmail, calendar, cv_sheet_manager, process_cvs, search_candidates, search_create_sheet, datetime, webex
✅ Loaded 8 tool(s)

Tool details:
  1. gmail
     Send and manage emails via Gmail API.

Operations:
- send_email: Send an email t...
  2. calendar
     Manage calendar events and schedules.

Operations:
- create_event: Create a new ...
  3. cv_sheet_manager
     Manage CV data in Google Sheets with full CRUD operations.

Operations:
- read_a...
  4. process_cvs
     Process all CVs from Google Drive folder and extract data to sheet.

Requires sh...
  5. search_candidates
     Search candidates in sheet and rank by job position match....
  6. search_create_sheet
     Search for sheet by name, create if not found. Returns sheet_id....
  7. datetime
     Get current date/time and perform time operations.

Operations:
- get_curren

## Step 2: Create and Configure LLM

Create the LLM and bind it to our tools

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from src.config import settings

print("Step 2: Creating LLM\n" + "="*60)

# Create the LLM
llm = ChatGoogleGenerativeAI(
    model=settings.MODEL_NAME,
    api_key=settings.GOOGLE_API_KEY,
    temperature=settings.TEMPERATURE
)

print(f"✅ LLM created")
print(f"   Model: {settings.MODEL_NAME}")
print(f"   Temperature: {settings.TEMPERATURE}")

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print(f"\n✅ Tools bound to LLM")
print(f"   {len(tools)} tool(s) available to the agent")

print("\n" + "="*60)
print("✅ LLM configured successfully!")

Step 2: Creating LLM
✅ LLM created
   Model: gemini-2.5-flash
   Temperature: 0.7

✅ Tools bound to LLM
   8 tool(s) available to the agent

✅ LLM configured successfully!


## Step 3: Define Agent State

Define the state that will be passed through the agent graph

In [7]:
from langgraph.graph import MessagesState
from typing import TypedDict

print("Step 3: Defining Agent State\n" + "="*60)

# Define custom state extending MessagesState
class AgentState(MessagesState):
    """Custom agent state with additional fields"""
    sender_phone: str  # WhatsApp phone number
    sender_identifier: str  # WhatsApp identifier

print("✅ Agent state defined")
print("\nState fields:")
print("  • messages: List[BaseMessage] - Conversation history")
print("  • sender_phone: str - User's phone number")
print("  • sender_identifier: str - WhatsApp identifier")

print("\n" + "="*60)
print("✅ State definition complete!")

Step 3: Defining Agent State
✅ Agent state defined

State fields:
  • messages: List[BaseMessage] - Conversation history
  • sender_phone: str - User's phone number
  • sender_identifier: str - WhatsApp identifier

✅ State definition complete!


## Step 4: Define Agent Logic

Create the functions that define the agent's behavior

In [8]:
from langchain_core.messages import SystemMessage
from langgraph.graph import END
from src.agents.prompts import SYSTEM_PROMPT

print("Step 4: Defining Agent Logic\n" + "="*60)

# Function 1: Call the model
def call_model(state: AgentState):
    """
    Call the LLM with the current conversation state.
    Adds system prompt if not present.
    """
    messages = state['messages']
    
    # Add system prompt if not present
    if not messages or not isinstance(messages[0], SystemMessage):
        messages = [SystemMessage(content=SYSTEM_PROMPT)] + messages
    
    # Call LLM
    response = llm_with_tools.invoke(messages)
    
    # Return new state with response
    return {"messages": [response]}

print("✅ Defined: call_model()")
print("   Purpose: Calls LLM and returns response")

# Function 2: Decide next step
def should_continue(state: AgentState):
    """
    Determine if the agent should continue or end.
    If the last message has tool calls, route to tools.
    Otherwise, end the conversation.
    """
    messages = state['messages']
    last_message = messages[-1]
    
    # If there are tool calls, continue to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    # Otherwise, end
    return END

print("\n✅ Defined: should_continue()")
print("   Purpose: Routes to tools or ends conversation")

print("\n" + "="*60)
print("✅ Agent logic defined successfully!")

Step 4: Defining Agent Logic
✅ Defined: call_model()
   Purpose: Calls LLM and returns response

✅ Defined: should_continue()
   Purpose: Routes to tools or ends conversation

✅ Agent logic defined successfully!


## Step 5: Build LangGraph Workflow

Assemble the agent graph with nodes and edges

In [9]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

print("Step 5: Building LangGraph Workflow\n" + "="*60)

# Create the graph
workflow = StateGraph(AgentState)

print("✅ Graph created\n")

# Add nodes
print("Adding nodes:")
workflow.add_node("agent", call_model)
print("  1. agent - Calls the LLM")

workflow.add_node("tools", ToolNode(tools))
print("  2. tools - Executes tool calls")

print("\nAdding edges:")

# Entry point
workflow.add_edge(START, "agent")
print("  • START → agent")

# Conditional edge from agent
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",  # If tools needed, go to tools
        END: END  # Otherwise, end
    }
)
print("  • agent → tools (if tool calls present)")
print("  • agent → END (if no tool calls)")

# Loop back from tools to agent
workflow.add_edge("tools", "agent")
print("  • tools → agent (for next iteration)")

print("\n" + "="*60)
print("✅ Workflow built successfully!")
print("\nGraph structure:")
print("  START → agent ⇄ tools")
print("            ↓")
print("           END")

Step 5: Building LangGraph Workflow
✅ Graph created

Adding nodes:
  1. agent - Calls the LLM
  2. tools - Executes tool calls

Adding edges:
  • START → agent
  • agent → tools (if tool calls present)
  • agent → END (if no tool calls)
  • tools → agent (for next iteration)

✅ Workflow built successfully!

Graph structure:
  START → agent ⇄ tools
            ↓
           END


## Step 6: Add Memory (Optional)

Compile the graph with PostgreSQL checkpointer for conversation memory

In [10]:
from src.memory.postgres import get_checkpointer

print("Step 6: Adding Memory\n" + "="*60)

# Option 1: Compile with memory
print("Compiling with PostgreSQL checkpointer...")
checkpointer = get_checkpointer()
agent_with_memory = workflow.compile(checkpointer=checkpointer)

print("✅ Agent compiled with memory")
print("   Conversations will persist across interactions")

# Option 2: Compile without memory (uncomment if needed)
print("\nAlternatively, compile without memory:")
print("  agent_without_memory = workflow.compile()")

print("\n" + "="*60)
print("✅ Memory configuration complete!")

Step 6: Adding Memory
Compiling with PostgreSQL checkpointer...
✅ LangGraph PostgreSQL checkpointer tables initialized
✅ Checkpointer ready with autocommit=True
✅ Agent compiled with memory
   Conversations will persist across interactions

Alternatively, compile without memory:
  agent_without_memory = workflow.compile()

✅ Memory configuration complete!


## Step 7: Test Your Custom Agent

Let's test the agent we just built!

In [12]:
from langchain_core.messages import HumanMessage, AIMessage
print("Step 7: Testing Custom Agent\n" + "="*60)

# Test 1: Simple query
print("\nTest 1: Simple query")
print("-" * 60)

query = "Hello! What can you help me with?"
print(f"Query: {query}\n")

# Must provide config with thread_id for checkpointer
config={'thread_id':'just'}

result = agent_with_memory.invoke({
    "messages": [HumanMessage(content=query)],
    "sender_phone": "custom_test_001",
    "sender_identifier": "test@example.com"
}, config=config)

print("Response:")
print(result['messages'][-1].content)

print("\n" + "="*60)
print("✅ Custom agent working!")

Step 7: Testing Custom Agent

Test 1: Simple query
------------------------------------------------------------
Query: list all mettings

Response:
I don't see any meetings scheduled in Webex. Would you like me to list events from your calendar instead, or perhaps create a new meeting?

✅ Custom agent working!


In [12]:
# Test 2: Query with tool
print("\nTest 2: Query requiring tool call")
print("-" * 60)

query = "What time is it now?"
print(f"Query: {query}\n")



result = agent_with_memory.invoke({
    "messages": [HumanMessage(content=query)],
    "sender_phone": "custom_test_002",
    "sender_identifier": "test@example.com"
}, config=config)

print("Response:")
print(result['messages'][-1].content)

print(f"\nTotal messages in conversation: {len(result['messages'])}")
print("\n" + "="*60)
print("✅ Tool calling works!")


Test 2: Query requiring tool call
------------------------------------------------------------
Query: What time is it now?

Response:
The current time is 19:25:08 UTC on Friday, October 31, 2025.

Total messages in conversation: 6

✅ Tool calling works!


In [13]:
# Test 3: Memory persistence
print("\nTest 3: Testing memory")
print("-" * 60)

thread_id = "custom_test_003"


# First message
print("First message:")
query1 = "My name is Charlie."
print(f"  Query: {query1}")

result1 = agent_with_memory.invoke({
    "messages": [HumanMessage(content=query1)],
    "sender_phone": thread_id,
    "sender_identifier": "charlie@example.com"
}, config=config)

print(f"  Response: {result1['messages'][-1].content}\n")

# Second message
print("Second message:")
query2 = "What's my name?"
print(f"  Query: {query2}")

result2 = agent_with_memory.invoke({
    "messages": [HumanMessage(content=query2)],
    "sender_phone": thread_id,
    "sender_identifier": "charlie@example.com"
}, config=config)

print(f"  Response: {result2['messages'][-1].content}")

print("\n" + "="*60)
print("✅ Memory persistence works!")


Test 3: Testing memory
------------------------------------------------------------
First message:
  Query: My name is Charlie.
  Response: Hello Charlie! It's nice to meet you. How can I assist you today with your HR recruitment tasks?

Second message:
  Query: What's my name?
  Response: Your name is Charlie.

✅ Memory persistence works!


## Step 8: Customize Your Agent

Learn how to customize the agent's behavior

### Customization 1: Change System Prompt

In [14]:
print("Customization 1: Custom System Prompt\n" + "="*60)

# Define custom prompt
CUSTOM_PROMPT = """
You are a friendly and enthusiastic recruiting assistant.
Always be positive and encouraging when talking to candidates.
Use emojis occasionally to make the conversation more engaging! 😊

Your main tasks:
- Help candidates with job applications
- Schedule interviews
- Answer questions about positions
"""

# Define new call_model function with custom prompt
def call_model_custom(state: AgentState):
    messages = state['messages']
    if not messages or not isinstance(messages[0], SystemMessage):
        messages = [SystemMessage(content=CUSTOM_PROMPT)] + messages
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Build new workflow with custom function
custom_workflow = StateGraph(AgentState)
custom_workflow.add_node("agent", call_model_custom)
custom_workflow.add_node("tools", ToolNode(tools))
custom_workflow.add_edge(START, "agent")
custom_workflow.add_conditional_edges(
    "agent", should_continue, {"tools": "tools", END: END}
)
custom_workflow.add_edge("tools", "agent")

custom_agent = custom_workflow.compile(checkpointer=get_checkpointer())

print("✅ Created agent with custom system prompt")
print("\nTest it:")


result = custom_agent.invoke({
    "messages": [HumanMessage(content="Hi, I'm interested in applying.")],
    "sender_phone": "custom_prompt_test",
    "sender_identifier": "test@example.com"
}, config=config)

print(result['messages'][-1].content)

Customization 1: Custom System Prompt
✅ Created agent with custom system prompt

Test it:
That's fantastic, Charlie! We're always excited to welcome new talent. To help you with your application, could you please tell me which position you're interested in? If you have a CV ready, I can also help you process it! 😊


### Customization 2: Select Specific Tools

In [15]:
print("\nCustomization 2: Specific Tools Only\n" + "="*60)

# Instead of using all tools, select specific ones
from src.tools.utilities.datetime_mcp import DateTimeMCPTool
from src.tools.google.gmail_mcp import GmailMCPTool

# Create specific tools
datetime_tool = DateTimeMCPTool()
gmail_tool = GmailMCPTool()

# Convert to LangChain tools
specific_tools = [
    datetime_tool.to_langchain_tool(),
    gmail_tool.to_langchain_tool()
]

print(f"✅ Using {len(specific_tools)} specific tools:")
for tool in specific_tools:
    print(f"  • {tool.name}")

# Create LLM with specific tools
llm_specific = llm.bind_tools(specific_tools)

# Build workflow (similar to before, but with specific tools)
print("\nYou can now build a workflow using only these specific tools!")



Customization 2: Specific Tools Only
✅ Using 2 specific tools:
  • datetime
  • gmail

You can now build a workflow using only these specific tools!


### Customization 3: Add Custom State Fields

In [16]:
print("\nCustomization 3: Custom State Fields\n" + "="*60)

# Define extended state
class ExtendedAgentState(MessagesState):
    sender_phone: str
    sender_identifier: str
    # NEW: Custom fields
    candidate_name: str = ""
    position_applied: str = ""
    interview_count: int = 0

print("✅ Created extended state with custom fields:")
print("  • candidate_name - Track candidate's name")
print("  • position_applied - Track position")
print("  • interview_count - Count interviews scheduled")

print("\nYou can now:")
print("  1. Access these fields in your agent functions")
print("  2. Update them based on conversation")
print("  3. Use them for custom logic")


Customization 3: Custom State Fields
✅ Created extended state with custom fields:
  • candidate_name - Track candidate's name
  • position_applied - Track position
  • interview_count - Count interviews scheduled

You can now:
  1. Access these fields in your agent functions
  2. Update them based on conversation
  3. Use them for custom logic


## Summary

Congratulations! You've learned how to build a custom LangGraph agent.

In [17]:
print("\n" + "="*80)
print("📚 CUSTOM AGENT TUTORIAL SUMMARY")
print("="*80)
print("")

print("✅ You learned how to:")
print("  1. Load and configure tools")
print("  2. Create and bind LLM with tools")
print("  3. Define agent state and logic")
print("  4. Build LangGraph workflow (nodes + edges)")
print("  5. Add memory with PostgreSQL checkpointer")
print("  6. Test your custom agent")
print("  7. Customize agent behavior")
print("")

print("🔧 Customization Options:")
print("  • System prompts - Change agent personality")
print("  • Tool selection - Choose which tools to use")
print("  • State fields - Add custom tracking fields")
print("  • Logic functions - Implement custom behaviors")
print("")

print("🎯 Key Concepts:")
print("  • StateGraph: Defines agent workflow")
print("  • Nodes: Processing steps (agent, tools)")
print("  • Edges: Flow control between nodes")
print("  • Checkpointer: Conversation memory")
print("  • Config: Must include thread_id for memory")
print("")

print("="*80)
print("🎉 YOU CAN NOW BUILD CUSTOM AGENTS!")
print("="*80)
print("")
print("Next Steps:")
print("1. Build your own agent for a specific use case")
print("2. Experiment with different tools and prompts")
print("3. Test MCP integration: See 04_mcp_integration.ipynb")
print("="*80)


📚 CUSTOM AGENT TUTORIAL SUMMARY

✅ You learned how to:
  1. Load and configure tools
  2. Create and bind LLM with tools
  3. Define agent state and logic
  4. Build LangGraph workflow (nodes + edges)
  5. Add memory with PostgreSQL checkpointer
  6. Test your custom agent
  7. Customize agent behavior

🔧 Customization Options:
  • System prompts - Change agent personality
  • Tool selection - Choose which tools to use
  • State fields - Add custom tracking fields
  • Logic functions - Implement custom behaviors

🎯 Key Concepts:
  • StateGraph: Defines agent workflow
  • Nodes: Processing steps (agent, tools)
  • Edges: Flow control between nodes
  • Checkpointer: Conversation memory
  • Config: Must include thread_id for memory

🎉 YOU CAN NOW BUILD CUSTOM AGENTS!

Next Steps:
1. Build your own agent for a specific use case
2. Experiment with different tools and prompts
3. Test MCP integration: See 04_mcp_integration.ipynb
